In [7]:
# Step: Test /predict from inside the same container (no external exposure needed)
import requests
import json
from pathlib import Path

# Adjust these paths if needed
BVH_PATH = "/workspace/bvh_model_project/spine_error.bvh"   # file you uploaded
API_URL = "http://127.0.0.1:8000/predict"                  # use 127.0.0.1 or 0.0.0.0

# Ensure file exists
p = Path(BVH_PATH)
if not p.exists():
    raise FileNotFoundError(f"BVH not found: {BVH_PATH}. Upload it to the container.")

# Send POST request with file upload
with open(BVH_PATH, "rb") as f:
    files = {"file": (p.name, f, "application/octet-stream")}
    print(f"Posting {p.name} to {API_URL} ...")
    resp = requests.post(API_URL, files=files, timeout=120)  # increase timeout if needed

# Print status and JSON
print("HTTP", resp.status_code)
try:
    result = resp.json()
    print(json.dumps(result, indent=2))
except Exception as e:
    print("Response is not JSON or parsing failed:", e)
    print(resp.text)

# Optional: save response to file
out_file = "/workspace/bvh_model_project/last_predict_response.json"
with open(out_file, "w") as out:
    json.dump({"status_code": resp.status_code, "response": resp.text}, out)
print("Saved response to", out_file)


Posting spine_error.bvh to http://127.0.0.1:8000/predict ...
HTTP 200
{
  "status": "success",
  "file": "spine_error.bvh",
  "analytics": {
    "total_frames": 8384,
    "class_frequencies": {
      "good": 0.505,
      "knee": 0.149,
      "spine": 0.346
    },
    "transitions": 182,
    "mean_confidence": 0.752,
    "prob_plot": "outputs/20251029_182313/prob_trend.png"
  },
  "feedback": "Your overall posture quality shows some areas for improvement, particularly with your spine and knees. The spine issues were noted frequently at a rate of 0.35, while knee issues were less frequent at 0.15. To enhance your posture, focus on strengthening your core and practicing spinal alignment exercises, which can help address those spine concerns. Additionally, incorporating stretching and strengthening for your knees will provide better support. Remember, small adjustments will lead to significant improvements\u2014keep up the great work!",
  "image_base64": null
}
Saved response to /workspace